# Import relevant libraries

In [1]:
import pandas as pd
import numpy as np

# Constants

In [55]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURES_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into Numpy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)



# Create a Full Matrix from a Sparse Matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword Arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of the occurence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(columns=column_names, index=doc_id_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
        
    full_matrix.set_index('DOC_ID', inplace=True)  
    return full_matrix


In [16]:
%%time

full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 18.3 s


# Probability of (Spam / Total Emails)

In [31]:
nr_of_spam = len(full_train_data[full_train_data['CATEGORY'] == 1])
prob_spam = nr_of_spam/full_train_data.shape[0]
print('Probability of spam is: ',prob_spam)

Probability of spam is:  0.31056829511465606


# Total Number of Words (Tokens)

### Counting Total number of Words (Tokens)

In [38]:
# Selecting all columns, except one
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']

# Sum all values in DataFrame
total_words = full_train_features.values.sum()
total_words

447611

# Total number of Words in Spam

In [36]:
full_spam_emails = full_train_data[full_train_data['CATEGORY'] == 1]

# Sum all NUMERIC values in DataFrame
# If data-frame contains non-numeric data, and want to just get sum of integers, do:
toal_words_spam = full_spam_emails.sum(numeric_only=True).sum()
toal_words_spam

193988

# Total number of Words in Ham

In [40]:
toal_words_ham = total_words - toal_words_spam
toal_words_ham

0.5666147614781585

# Sum the tokens accross Spam and Ham Emails 

## Sum the total of each token in Spam Subset

In [44]:
full_spam_emails = full_train_data[full_train_data['CATEGORY'] == 1]

# We have to add +1 to avoid division of 0
summed_spam_tokens = full_spam_emails.sum(axis=0)+1
summed_spam_tokens

CATEGORY    1247
0           2213
1           1010
2           1366
3           2049
            ... 
2495          12
2496           1
2497          26
2498           1
2499           3
Length: 2501, dtype: int64

## Sum the total of each token in Ham Subset

In [45]:
full_ham_emails = full_train_data[full_train_data['CATEGORY'] == 0]

# We have to add +1 to avoid division of 0
summed_ham_tokens = full_ham_emails.sum() + 1
summed_ham_tokens

CATEGORY       1
0           5407
1           2673
2           2067
3            909
            ... 
2495          18
2496          28
2497          11
2498           4
2499          40
Length: 2501, dtype: int64

# P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [46]:
# Cuz we add +1 in summed_spam_tokens, hence we are getting 1 more word for each token
# => we're having 2500 more words in the numerator 
# => so we must add +2500 words to the denominator
prob_tokens_spam = summed_spam_tokens / (toal_words_spam + VOCAB_SIZE) 
prob_tokens_spam

CATEGORY    0.006346
0           0.011263
1           0.005140
2           0.006952
3           0.010428
              ...   
2495        0.000061
2496        0.000005
2497        0.000132
2498        0.000005
2499        0.000015
Length: 2501, dtype: float64

# P(Token | Ham) - Probability that a Token Occurs given the Email is NonSpam

In [48]:
# Cuz we add +1 in summed_spam_tokens, hence we are getting 1 more word for each token
# => we're having 2500 more words in the numerator 
# => so we must add +2500 words to the denominator
prob_tokens_nonspam = summed_ham_tokens / (toal_words_ham + VOCAB_SIZE) 
prob_tokens_nonspam

CATEGORY    0.000004
0           0.021111
1           0.010436
2           0.008070
3           0.003549
              ...   
2495        0.000070
2496        0.000109
2497        0.000043
2498        0.000016
2499        0.000156
Length: 2501, dtype: float64

# P(Token | Ham) - Probability that a Token Occurs

In [51]:
prob_tokens_all = full_train_features.sum(axis=0)/total_words
prob_tokens_all

0       0.017019
1       0.008224
2       0.007665
3       0.006604
4       0.006792
          ...   
2495    0.000063
2496    0.000060
2497    0.000078
2498    0.000007
2499    0.000092
Length: 2500, dtype: float64

# Save the Trained Model

In [54]:
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [56]:
%%time

full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 6.19 s


In [57]:
full_test_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,5,5,2,2,1,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5790,0,2,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,4,0,2,4,3,3,1,4,...,0,0,0,0,0,2,0,0,0,0


In [64]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [65]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURES_MATRIX, X_test)